# Red Neuronal Convolucional - EfficientNet-B0 | 5.3 M Parametros

## Inputs

### Instalación de librerias

In [ ]:
!pip install timm torchmetrics albumentations opencv-python astropy grad-cam


### Librerias

In [ ]:
import os, math, random, json, glob
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
import cv2
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import average_precision_score, f1_score
from torchmetrics.classification import BinaryAUROC, BinaryCalibrationError
import timm
from astropy.io import fits
import cv2, numpy as np, pandas as pd
from torch.utils.data import Dataset
from astropy.io import fits
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import BinaryClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import numpy as np
import pandas as pd
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    average_precision_score,
    roc_auc_score
)
from torch.utils.data import DataLoader
import torch



### SEED y GPU

In [ ]:
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Conexión con Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Procesamiento de Imagenes

### Procesamiento y apilado de imágenes astronómicas

**Descripción**

El código proporciona dos funciones principales:
- **read_band_image**: Lee imágenes astronómicas en formato FITS o PNG, normaliza los valores en el rango [0,1] y aplica un estirado asinh para mejorar el contraste de forma suave.
- **stack_bands**: Toma varias bandas (por defecto `g, r, z`), las centra y recorta a un formato cuadrado, las redimensiona a un tamaño fijo y finalmente las apila en un tensor con la forma C×H×W.

**Técnicas utilizadas**

- **Normalización por percentiles** (1%–99.5%) para limitar el rango dinámico y eliminar valores extremos.
- **Transformación asinh** con ganancia ≈10 para un contraste suave y balanceado entre regiones brillantes y débiles.
- **Conversión a escala de grises** para imágenes en color.
- **Centrado y padding a cuadrado** antes del escalado para evitar distorsiones geométricas.
- **Redimensionamiento conservador** con interpolación `INTER_AREA` para mantener la calidad en la reducción de tamaño.
- **Apilado de canales** con `np.stack` para generar un tensor multicanal adecuado para modelos de visión por computadora.

**Justificación**

- El **recorte por percentiles** evita que valores atípicos dominen la escala de intensidad.
- El **estirado asinh** es una técnica común en astrofotografía para resaltar estructuras débiles sin saturar las regiones brillantes.
- El **padding centrado** garantiza que la relación de aspecto se preserve antes del resize, evitando deformaciones.
- La **interpolación adecuada** mantiene la fidelidad visual al reducir la resolución.
- El **apilado en formato tensorial** prepara los datos para su uso en redes neuronales o pipelines de análisis de imágenes científicas.

In [ ]:
def read_band_image(path):
    # Lee FITS o PNG y devuelve float32 en [0,1] con asinh stretch controlado
    if path.endswith(".fits"):
        data = fits.getdata(path).astype(np.float32)
        data = np.nan_to_num(data, nan=0.0, posinf=0.0, neginf=0.0)
    else:
        data = cv2.imread(path, cv2.IMREAD_UNCHANGED).astype(np.float32)
        if data.ndim==3: data = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
    # recorte de percentiles y asinh
    lo, hi = np.percentile(data, [1, 99.5])
    if hi<=lo: hi = lo+1e-6
    data = np.clip((data - lo) / (hi - lo), 0, 1)
    data = np.arcsinh(10*data) / np.arcsinh(10)  # contraste suave
    return data

def stack_bands(root, image_id, bands=("g","r","z"), ext=".fits", size=256):
    chans = []
    for b in bands:
        p = f"{root}/{b}/{image_id}{ext}"
        if not os.path.exists(p):
            p = f"{root}/{b}/{image_id}.png"
        img = read_band_image(p)
        # centra/pad a cuadrado y resize conservador
        h,w = img.shape
        m = max(h,w)
        canvas = np.zeros((m,m), np.float32)
        y0 = (m-h)//2; x0 = (m-w)//2
        canvas[y0:y0+h, x0:x0+w] = img
        img = cv2.resize(canvas, (size,size), interpolation=cv2.INTER_AREA)
        chans.append(img)
    x = np.stack(chans, axis=0)  #
    return x


### Dataset personalizado con imágenes astronómicas y Data Augmentation

**Descripción**

Este código define un conjunto de utilidades y una clase `BarsDatasetPaths` para construir un **dataset de imágenes astronómicas** a partir de rutas explícitas en un archivo CSV.  
Incluye funciones auxiliares para:
- Leer imágenes FITS/PNG en escala de grises.
- Aplicar normalización de contraste mediante estirado asinh.
- Centrar, acolchar y redimensionar las imágenes a un tamaño uniforme.
- Apilar las bandas g, r, z en tensores.
Además, integra **transformaciones de data augmentation** (rotaciones, traslaciones, ruido, etc.) usando la librería Albumentations, y devuelve tensores junto con las etiquetas.

**Técnicas utilizadas**

- **Lectura robusta de FITS y PNG** con saneamiento de valores (NaN/Inf → 0).
- **Normalización por percentiles (1–99.5%)** para estabilizar el rango dinámico.
- **Transformación asinh** para resaltar detalles débiles sin saturar zonas brillantes.
- **Centrado y padding a cuadrado + resize** para homogenizar dimensiones de entrada.
- **Apilado de canales** g/r/z en formato tensorial.
- **Data augmentation con Albumentations**: rotaciones, traslaciones, desenfoque gaussiano, ruido gaussiano, y ajuste de brillo/contraste.
- **Conversión a tensores PyTorch** con `ToTensorV2`.

**Justificación**

- La **lectura multi-formato (FITS/PNG)** garantiza compatibilidad con diferentes fuentes de datos astronómicos.
- El **recorte por percentiles y estirado asinh** son técnicas estándar en astrofotografía para mejorar la visibilidad de estructuras astronómicas.
- El **padding y resize** evitan deformaciones geométricas y permiten entrenar modelos con entradas de tamaño fijo.
- El **apilado de bandas g/r/z** conserva la información multicanal relevante para análisis de galaxias y barras estelares.
- El uso de **augmentaciones** introduce variaciones realistas en los datos de entrenamiento, mejorando la capacidad de generalización del modelo.
- El diseño en forma de **Dataset de PyTorch** permite integrarse fácilmente en pipelines de entrenamiento con `DataLoader`.

In [ ]:

def _read_fits_first2d(path):
    with fits.open(path, memmap=False) as hdul:
        for h in hdul:
            if h.data is not None and getattr(h.data, "ndim", 0)==2:
                arr = h.data.astype(np.float32); break
    return np.nan_to_num(arr, nan=0.0, posinf=0.0, neginf=0.0)

def _read_any_gray(path):
    p = str(path).lower()
    if p.endswith(".fits") or p.endswith(".fz"):
        return _read_fits_first2d(path)
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img is None: raise FileNotFoundError(path)
    if img.ndim==3: img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img.astype(np.float32)

def _stretch_asinh(x):
    lo, hi = np.percentile(x, [1, 99.5])
    x = np.clip((x-lo)/(hi-lo+1e-6), 0, 1)
    return np.arcsinh(10*x)/np.arcsinh(10)

def _pad_resize_square(img, size=256):
    h,w = img.shape; m = max(h,w)
    canvas = np.zeros((m,m), np.float32)
    y0=(m-h)//2; x0=(m-w)//2
    canvas[y0:y0+h, x0:x0+w] = img
    return cv2.resize(canvas, (size,size), interpolation=cv2.INTER_AREA)

def stack_from_row(row, size=256):
    chans=[]
    for b in ["g","r","z"]:
        p = row[f"path_{b}"]
        img = _read_any_gray(p)
        img = _pad_resize_square(_stretch_asinh(img), size)
        chans.append(img)
    return np.stack(chans, axis=0)  # CxHxW

def build_transforms(train=True):
    if train:
        return A.Compose([
            A.Rotate(limit=180, border_mode=cv2.BORDER_REFLECT_101, p=1.0),
            A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.0, rotate_limit=0, border_mode=cv2.BORDER_REFLECT_101, p=0.5),
            A.GaussianBlur(blur_limit=(3,5), p=0.3),
            A.GaussNoise(var_limit=(1e-5,5e-4), p=0.3),
            A.RandomBrightnessContrast(0.05,0.05,p=0.3),
            ToTensorV2()
        ])
    else:
        return A.Compose([ToTensorV2()])

class BarsDatasetPaths(Dataset):
    def __init__(self, csv_path, size=256, train=True):
        self.df = pd.read_csv(csv_path)
        self.size = size
        self.tfm = build_transforms(train)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        r = self.df.iloc[i]
        x = stack_from_row(r, size=self.size)
        x = np.transpose(x, (1,2,0))
        x = self.tfm(image=x)["image"]
        y_bin = torch.tensor(r.label_bin, dtype=torch.float32)
        y_str = torch.tensor(r.Bars,      dtype=torch.float32)
        return x, y_bin, y_str, r.image_id


In [ ]:
ds = BarsDatasetPaths("/content/drive/MyDrive/Proyecto_Integrador/Deteccion/datasets/train_grz.csv", size=256, train=True)
x, yb, ys, _ = ds[0]
print(x.shape)  # debe ser torch.Size([3, 256, 256])


## Red Neuronal

BarNet en PyTorch con EfficientNet como backbone y dos cabezas de predicción (binaria y continua)

**Descripción**

La clase `BarNet` implementa un modelo de deep learning en PyTorch diseñado para analizar imágenes astronómicas multibanda.  
Se basa en un **backbone EfficientNet** (de la librería `timm`) para la extracción de características y añade dos cabezas totalmente conectadas:  
- `head_bin`: salida binaria (detección de presencia o ausencia de barras).  
- `head_str`: salida continua (estimación de la fuerza/intensidad de la barra).  

El método `forward` obtiene las características del backbone y las procesa en paralelo por ambas cabezas para producir predicciones complementarias.

**Técnicas utilizadas**

- **Transfer learning con EfficientNet**: uso de `timm.create_model` con pesos preentrenados y adaptación de la entrada a 3 canales.  
- **Regularización** mediante `Dropout` y `DropPath` para reducir sobreajuste.  
- **Red neuronal secuencial** para las cabezas de predicción, con capas lineales, activación `ReLU` y dropout intermedio.  
- **Diseño multitarea**: dos salidas diferentes desde el mismo espacio de características (`logit` binario y `strength` continuo).  

**Justificación**

- El uso de **EfficientNet como backbone** aprovecha un modelo eficiente y potente para extraer representaciones visuales profundas de las imágenes astronómicas.  
- La **cabeza binaria** permite entrenar el modelo para una clasificación sencilla (¿existe o no una barra galáctica?).  
- La **cabeza continua** añade información más rica, cuantificando la intensidad de la barra, lo cual es útil para análisis más finos en astrofísica.  
- La **regularización con Dropout** y el uso de pesos preentrenados mejoran la generalización, evitando sobreajuste y acelerando la convergencia.  
- El **diseño multitarea** en un solo modelo es más eficiente y permite compartir representaciones entre tareas relacionadas.  


In [ ]:
class BarNet(nn.Module):
    def __init__(self, backbone="tf_efficientnet_b0_ns", in_chans=3, drop=0.2):
        super().__init__()
        self.backbone = timm.create_model(backbone, pretrained=True, in_chans=in_chans, drop_rate=drop, drop_path_rate=0.1, num_classes=0, global_pool="avg")
        emb = self.backbone.num_features
        self.head_bin = nn.Sequential(nn.Linear(emb, 256), nn.ReLU(), nn.Dropout(0.2), nn.Linear(256,1))
        self.head_str = nn.Sequential(nn.Linear(emb, 256), nn.ReLU(), nn.Dropout(0.2), nn.Linear(256,1))
    def forward(self, x):
        feat = self.backbone(x)
        logit = self.head_bin(feat).squeeze(1)
        strength = self.head_str(feat).squeeze(1)
        return logit, strength


### Pérdidas, Métricas y ciclos de entrenamiento/validación

**Descripción**

Este bloque de código implementa:
- **`losses_and_metrics`**: calcula la función de pérdida combinada (clasificación binaria + regresión de fuerza) y métricas rápidas de evaluación por batch.
- **`train_one_epoch`**: ejecuta un ciclo de entrenamiento para una época, incluyendo cálculo de pérdidas, retropropagación, clipping de gradientes y optimización.
- **`validate`**: evalúa el modelo sin gradientes sobre un conjunto de validación, acumulando predicciones para calcular métricas globales.

**Técnicas utilizadas**

- **Binary Cross-Entropy (BCE)** con logits para la tarea de clasificación binaria (detección de barras).
- **Huber Loss** aplicada a la salida de fuerza (tras sigmoid) para manejar regresión robusta en el rango [0,1].
- **Combinación ponderada de pérdidas**: `loss = BCE + 0.5*Huber`.
- **Métricas de clasificación**: AUPRC (área bajo la curva de precisión-recall) y F1-score con umbral 0.5.
- **Métrica de regresión**: error absoluto medio (MAE) para la fuerza de la barra.
- **Entrenamiento supervisado estándar** en PyTorch: forward → loss → backward → optimización.
- **Regularización mediante gradient clipping** para evitar explosión de gradientes.
- **Validación sin gradientes** (`@torch.no_grad()`) para reducir memoria y acelerar inferencia.

**Justificación**

- La combinación de **BCE** y **Huber** refleja el enfoque multitarea del modelo: detección (binaria) y cuantificación (continua).
- El **Huber Loss** es más robusto frente a outliers que el MSE, lo que resulta adecuado en datos astronómicos con valores extremos.
- El cálculo de **AUPRC** es crítico en datasets desbalanceados, mientras que el **F1-score** aporta una métrica intuitiva en clasificación.
- El **MAE** ofrece una medida interpretable de error medio en la estimación de fuerza.
- El **clipping de gradientes** estabiliza el entrenamiento de redes profundas evitando actualizaciones inestables.
- La separación de funciones (`train_one_epoch` vs `validate`) promueve código limpio, modular y fácil de mantener en pipelines de entrenamiento.


In [ ]:
def losses_and_metrics(logit, strength_pred, y_bin, y_str):
    bce = F.binary_cross_entropy_with_logits(logit, y_bin)
    huber = F.huber_loss(torch.sigmoid(strength_pred), y_str)
    loss = bce + 0.5*huber
    prob = torch.sigmoid(logit).detach().cpu().numpy()
    yb = y_bin.detach().cpu().numpy()
    auprc = average_precision_score(yb, prob) if (yb.min()!=yb.max()) else np.nan
    f1 = f1_score((prob>=0.5).astype(int), yb.astype(int)) if (yb.min()!=yb.max()) else np.nan
    mae = np.mean(np.abs(torch.sigmoid(strength_pred).detach().cpu().numpy() - y_str.detach().cpu().numpy()))
    return loss, {"auprc":auprc, "f1@0.5":f1, "mae_str":mae}

def train_one_epoch(model, loader, opt):
    model.train()
    logs=[]
    for x, yb, ys, _ in loader:
        x, yb, ys = x.to(device), yb.to(device), ys.to(device)
        opt.zero_grad()
        logit, sp = model(x)
        loss, _ = losses_and_metrics(logit, sp, yb, ys)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 2.0)
        opt.step()
        logs.append(loss.item())
    return float(np.mean(logs))

@torch.no_grad()
def validate(model, loader):
    model.eval()
    probs=[]; gts=[]; preds_str=[]; gts_str=[]
    for x, yb, ys, _ in loader:
        x = x.to(device)
        logit, sp = model(x)
        probs.append(torch.sigmoid(logit).cpu().numpy())
        preds_str.append(torch.sigmoid(sp).cpu().numpy())
        gts.append(yb.numpy()); gts_str.append(ys.numpy())
    prob = np.concatenate(probs); yb = np.concatenate(gts).astype(np.float32)
    sp = np.concatenate(preds_str); ys = np.concatenate(gts_str).astype(np.float32)
    auprc = average_precision_score(yb, prob) if (yb.min()!=yb.max()) else float("nan")
    f1 = f1_score((prob>=0.5).astype(int), yb.astype(int)) if (yb.min()!=yb.max()) else float("nan")
    mae = float(np.mean(np.abs(sp - ys)))
    return {"val_auprc":auprc, "val_f1@0.5":f1, "val_mae_str":mae}


### Función de Entrenamiento

**Descripción**

La función `fit_paths` implementa el ciclo de entrenamiento y validación para el modelo **BarNet** a partir de rutas CSV con datasets de entrenamiento y validación.  
Su flujo principal incluye:
1. Construcción de datasets (`BarsDatasetPaths`) y `DataLoader` para batching eficiente.
2. Creación del modelo `BarNet` con un backbone configurable.
3. Configuración del optimizador (`AdamW`) y scheduler de tasa de aprendizaje (`CosineAnnealingLR`).
4. Ejecución de múltiples épocas de entrenamiento con:
   - Forward y backward pass.
   - Cálculo de pérdidas y métricas.
   - Guardado de checkpoints (`last.pt` y `best.pt`).
   - Estrategia de **early stopping** basada en métricas de validación.
5. Retorno de la ruta al mejor modelo guardado.

**Técnicas utilizadas**

- **PyTorch DataLoader** con *multi-threading* (`num_workers=4`) y `pin_memory` para acelerar transferencia a GPU.
- **AdamW** como optimizador con regularización L2 vía `weight_decay`.
- **Cosine Annealing LR** para variar la tasa de aprendizaje de forma suave durante las épocas.
- **Validación periódica** con métricas (`val_auprc`, `val_f1`, `val_mae_str`).
- **Checkpointing** automático de los modelos (`last.pt` y `best.pt`) para reproducibilidad.
- **Early stopping** con paciencia configurable (6 épocas) para evitar sobreentrenamiento.

**Justificación**

- La construcción de datasets a partir de **CSV explícitos** permite trazabilidad de los datos de entrenamiento/validación.
- El uso de **AdamW** mejora la convergencia en visión por computadora frente al Adam estándar, especialmente con regularización adecuada.
- El **CosineAnnealingLR** proporciona un esquema de ajuste dinámico del learning rate que favorece estabilidad y evita mínimos locales.
- El **checkpointing** asegura la preservación de los mejores pesos incluso si el entrenamiento se interrumpe.
- La técnica de **early stopping** ahorra tiempo computacional y evita sobreajuste al detener el entrenamiento cuando el modelo deja de mejorar en validación.
- La función devuelve la ruta al mejor modelo (`best.pt`), facilitando su carga directa en fases posteriores de evaluación o inferencia.


In [ ]:
def fit_paths(csv_train, csv_val, size=256, in_chans=3, epochs=20, lr=2e-4, bs=32,
              backbone="tf_efficientnet_b0_ns", out_dir="runs/barnet"):
    os.makedirs(out_dir, exist_ok=True)
    tr_ds = BarsDatasetPaths(csv_train, size=size, train=True)
    va_ds = BarsDatasetPaths(csv_val,   size=size, train=False)
    tr = DataLoader(tr_ds, batch_size=bs, shuffle=True,  num_workers=4, pin_memory=True)
    va = DataLoader(va_ds, batch_size=bs, shuffle=False, num_workers=4, pin_memory=True)

    model = BarNet(backbone=backbone, in_chans=in_chans).to(device)
    opt = AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    sch = CosineAnnealingLR(opt, T_max=epochs)
    best = -1; patience=6; bad=0

    for ep in range(1, epochs+1):
        tr_loss = train_one_epoch(model, tr, opt)
        sch.step()
        m = validate(model, va)
        score = m["val_auprc"]
        torch.save({"model":model.state_dict(),"epoch":ep,"metrics":m}, f"{out_dir}/last.pt")
        if score>best:
            best=score; bad=0
            torch.save({"model":model.state_dict(),"epoch":ep,"metrics":m}, f"{out_dir}/best.pt")
        else:
            bad += 1
        print(f"[{ep}/{epochs}] loss {tr_loss:.4f} | {m}")
        if bad>=patience:
            print("Early stop."); break
    return f"{out_dir}/best.pt"


### Threshold para clasificación binaria

**Descripción**

La función `pick_threshold` evalúa un modelo entrenado (`BarNet`) sobre un conjunto de datos para encontrar el **umbral de probabilidad** que maximiza el **F1-score** en la tarea binaria de detección de barras.  
El flujo es el siguiente:
1. Ejecuta el modelo en modo evaluación (`eval`) y desactiva el cálculo de gradientes (`@torch.no_grad`).
2. Recolecta las probabilidades predichas (`sigmoid(logit)`) y las etiquetas reales.
3. Recorre una grilla de posibles umbrales en el rango [0.2, 0.8].
4. Calcula el **F1-score** para cada umbral.
5. Devuelve el mejor umbral y su correspondiente valor de F1.

**Técnicas utilizadas**

- **Inferencia sin gradientes** (`torch.no_grad`) para optimizar memoria y velocidad.
- **Predicción de probabilidades** mediante la función sigmoide aplicada a logits.
- **Evaluación sistemática de umbrales** con búsqueda en rejilla (grid search) en pasos uniformes.
- **Métrica F1-score** como criterio de selección para balancear precisión y exhaustividad.
- **Concatenación de batches** en arrays NumPy para el cálculo global de métricas.

**Justificación**

- El umbral por defecto (0.5) puede no ser óptimo en datasets desbalanceados; este método ajusta el punto de decisión al conjunto de validación.
- La búsqueda en el rango [0.2–0.8] cubre escenarios donde las predicciones son más conservadoras o más liberales, permitiendo adaptar el modelo a las necesidades específicas.
- El **F1-score** se utiliza porque equilibra precisión y recall, lo cual es crítico en problemas donde detectar correctamente las barras astronómicas es más importante que minimizar un tipo específico de error.
- Seleccionar un **umbral óptimo validado** mejora la aplicabilidad práctica del modelo y la calidad de las predicciones en producción.


In [ ]:
@torch.no_grad()
def pick_threshold(model, loader):
    model.eval()
    probs=[]; gts=[]
    for x, yb, _, _ in loader:
        x = x.to(device)
        logit, _ = model(x)
        probs.append(torch.sigmoid(logit).cpu().numpy())
        gts.append(yb.numpy())
    p = np.concatenate(probs); y = np.concatenate(gts).astype(int)
    best_f1, best_t = -1, 0.5
    for t in np.linspace(0.2,0.8,25):
        f1 = f1_score((p>=t).astype(int), y)
        if f1>best_f1: best_f1, best_t = f1, t
    return best_t, best_f1


### Interpretación de Predicciones

**Descripción**

La función `gradcam_overlay` genera un **mapa de activación Grad-CAM** superpuesto sobre una imagen astronómica para interpretar qué regiones han influido en la decisión del modelo `BarNet`.  
El flujo es el siguiente:
1. Pone el modelo en modo evaluación.
2. Selecciona la capa objetivo para Grad-CAM:
   - Por defecto, la capa final del *backbone*.
   - Como alternativa, la última convolución disponible.
3. Calcula los mapas de activación Grad-CAM para la clase positiva (“con barra”).
4. Normaliza la imagen de entrada a rango [0,1].
5. Superpone el mapa de calor sobre la imagen en color para obtener una visualización interpretativa.

**Técnicas utilizadas**

- **Grad-CAM (Gradient-weighted Class Activation Mapping)** para identificar regiones relevantes de la imagen en la predicción.
- **Selección dinámica de la capa objetivo**: global pooling o última convolución.
- **Normalización de la entrada** a rango [0,1] para visualización adecuada.
- **Superposición de mapas de calor** (`show_cam_on_image`) para resaltar zonas de mayor activación.
- **Target específico de clase** mediante `BinaryClassifierOutputTarget`.

**Justificación**

- Grad-CAM proporciona **interpretabilidad** al modelo, mostrando en qué regiones de la galaxia se centra para detectar barras.
- La opción de elegir la **última capa convolucional** garantiza robustez, incluso si el *backbone* cambia.
- La **normalización y superposición** permiten producir visualizaciones claras y comparables entre distintas imágenes.
- El enfoque orientado a la clase positiva (“con barra”) es consistente con el objetivo científico: localizar evidencias de estructuras de barra en galaxias.
- Este tipo de visualización es esencial para validar que el modelo **aprende patrones astronómicamente relevantes** y no artefactos de los datos.


In [ ]:


def gradcam_overlay(model, x_tensor, target_layer=None):
    model.eval()
    if target_layer is None:
        # capa final del backbone
        target_layer = [model.backbone.get_global_pool().flatten]
        # fallback: usa la última conv si el backbone lo expone
        target_layer = [list(model.backbone.modules())[-2]]
    cam = GradCAM(model=model, target_layers=target_layer, use_cuda=(device.type=="cuda"))
    targets = [BinaryClassifierOutputTarget(1)]  # clase "con barra"
    grayscale_cam = cam(input_tensor=x_tensor, targets=targets)[0]
    img = x_tensor[0].permute(1,2,0).cpu().numpy()
    img = (img - img.min())/(img.max()-img.min()+1e-8)
    vis = show_cam_on_image(img, grayscale_cam, use_rgb=True)
    return vis  # array RGB


## Ejecución de Entrenamiento

- Tamaño de las imágenes (`256x256`).
- Número de canales de entrada (`3`).
- Número de épocas (`30`).
- Tasa de aprendizaje (`2e-4`).
- Batch size (`32`).
- Backbone (`tf_efficientnet_b0_ns`).
- Directorio de salida para checkpoints y logs.  

In [ ]:
DATASETS_DIR = "/content/drive/MyDrive/Proyecto_Integrador/Deteccion/datasets"
ckpt = fit_paths(
    csv_train=f"{DATASETS_DIR}/train_grz.csv",
    csv_val  =f"{DATASETS_DIR}/val_grz.csv",
    size=256,
    in_chans=3,
    epochs=30,
    lr=2e-4,
    bs=32,
    backbone="tf_efficientnet_b0_ns",
    out_dir="/content/drive/MyDrive/Proyecto_Integrador/Deteccion/runs/barnet"
)


## Evaluación final del modelo BarNet


Conjunto de test con métricas de clasificación, regresión y análisis de errores

**Descripción**

Este bloque de código realiza la **evaluación completa del modelo BarNet** usando el conjunto de test.  
El flujo incluye:
1. **Carga del mejor modelo** (`best.pt`) con `weights_only=False` para restaurar pesos y estado del entrenamiento.
2. **Construcción del DataLoader de test** a partir de `test_grz.csv`.
3. **Predicción en inferencia** (`torch.no_grad`) para obtener:
   - Probabilidades (`probs`) y etiquetas predichas (`y_pred`).
   - Valores continuos de fuerza predicha (`strengths_pred`) y reales (`strengths_true`).
   - Identificadores de imagen (`image_ids`).
4. **Cálculo de métricas finales**:
   - Clasificación: Accuracy, Precisión, Recall, F1-score, AUPRC, AUROC.
   - Regresión: Error absoluto medio (MAE) en la fuerza de la barra.
   - Reporte de clasificación y matriz de confusión.
5. **Guardado de resultados detallados** en un CSV (`test_results.csv`) con etiquetas reales, predicciones, probabilidades y errores.
6. **Análisis de errores**: conteo de falsos positivos, falsos negativos y proporción total de errores.

**Técnicas utilizadas**

- **Carga de checkpoints** con `torch.load` y `load_state_dict` para restaurar el modelo entrenado.
- **Inferencia eficiente** con `torch.no_grad` para desactivar gradientes.
- **Evaluación de clasificación binaria** con métricas estándar: clasificación report, matriz de confusión, accuracy, precision, recall, F1, AUPRC y AUROC.
- **Evaluación de regresión** mediante MAE sobre la predicción de fuerza.
- **Análisis post-evaluación** separando errores en falsos positivos y falsos negativos.
- **Exportación de resultados** a CSV para trazabilidad y análisis posterior.

**Justificación**

- La **carga segura del modelo** garantiza reproducibilidad de los resultados sin necesidad de reentrenar.
- La **evaluación en test** proporciona una medida objetiva del desempeño final, independiente de los datos usados en entrenamiento y validación.
- El uso conjunto de **métricas de clasificación y regresión** refleja el carácter multitarea del modelo: detección binaria y cuantificación de fuerza.
- La **matriz de confusión** permite entender los tipos de errores (FP, FN), crucial para valorar la utilidad científica del modelo.
- Guardar las predicciones en un **CSV detallado** facilita auditoría, análisis de casos problemáticos y futuras visualizaciones.
- El **análisis de errores** da transparencia al desempeño del modelo, identificando en qué casos falla más y guiando mejoras futuras.


In [ ]:
# 1. Cargar modelo con weights_only=False
DATASETS_DIR = "/content/drive/MyDrive/Proyecto_Integrador/Deteccion/datasets"
BEST = "/content/drive/MyDrive/Proyecto_Integrador/Deteccion/runs/barnet/best.pt"

# Cargar checkpoint
ckpt = torch.load(BEST, map_location=device, weights_only=False)
model.load_state_dict(ckpt["model"])
model.eval()

print("Modelo cargado exitosamente")

# 2. Preparar test loader
TEST_CSV = f"{DATASETS_DIR}/test_grz.csv"
test_ds = BarsDatasetPaths(TEST_CSV, size=256, train=False)
test_dl = DataLoader(test_ds, batch_size=64, num_workers=2, pin_memory=True)

# 3. Predecir
probs = []
y_true = []
strengths_pred = []
strengths_true = []
image_ids = []

with torch.no_grad():
    for x, yb, ys, img_id in test_dl:
        x = x.to(device)
        logits, sp = model(x)

        probs.append(torch.sigmoid(logits).cpu().numpy())
        y_true.append(yb.numpy())
        strengths_pred.append(torch.sigmoid(sp).cpu().numpy())
        strengths_true.append(ys.numpy())
        image_ids.extend(img_id)

probs = np.concatenate(probs)
y_true = np.concatenate(y_true).astype(int)
y_pred = (probs >= 0.5).astype(int)
strengths_pred = np.concatenate(strengths_pred)
strengths_true = np.concatenate(strengths_true)

# 4. METRICAS FINALES
print("="*60)
print("           RESULTADOS EN TEST SET")
print("="*60)

# Clasificacion
print("\nReporte de Clasificacion:")
print(classification_report(y_true, y_pred,
                          target_names=["Sin barra (0)", "Con barra (1)"],
                          digits=4))

# Matriz de confusion
print("\nMatriz de Confusion:")
cm = confusion_matrix(y_true, y_pred)
print(f"                Pred: Sin barra | Con barra")
print(f"Real: Sin barra       {cm[0,0]:4d}        {cm[0,1]:4d}")
print(f"      Con barra       {cm[1,0]:4d}        {cm[1,1]:4d}")

tn, fp, fn, tp = cm.ravel()
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print(f"\nMetricas Agregadas:")
print(f"  Accuracy:  {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"  F1-Score:  {f1:.4f}")
print(f"  AUPRC:     {average_precision_score(y_true, probs):.4f}")
print(f"  AUROC:     {roc_auc_score(y_true, probs):.4f}")

# Error en fuerza
mae_strength = np.mean(np.abs(strengths_pred - strengths_true))
print(f"  MAE (fuerza): {mae_strength:.4f}")

# 5. Guardar predicciones
results = pd.DataFrame({
    'image_id': image_ids,
    'true_label': y_true,
    'pred_prob': probs,
    'pred_label': y_pred,
    'true_strength': strengths_true,
    'pred_strength': strengths_pred,
    'correct': y_true == y_pred,
    'error': np.abs(y_true - y_pred)
})

OUT_CSV = f"{DATASETS_DIR}/test_results.csv"
results.to_csv(OUT_CSV, index=False)
print(f"\nGuardado: {OUT_CSV}")

# 6. Analisis de errores
errors = results[~results['correct']]
print(f"\nErrores: {len(errors)}/{len(results)} ({100*len(errors)/len(results):.2f}%)")

fp_errors = errors[errors['true_label'] == 0]
print(f"  Falsos Positivos: {len(fp_errors)}")

fn_errors = errors[errors['true_label'] == 1]
print(f"  Falsos Negativos: {len(fn_errors)}")

print("\nEvaluacion completa.")